In [ ]:
!pip install camelot-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 40 kB 23 kB/s 
     |████████████████████████████████| 5.6 MB 7.8 MB/s 
     |████████████████████████████████| 88 kB 6.4 MB/s 
     |████████████████████████████████| 4.0 MB 8.4 MB/s 


In [ ]:
!apt install ghostscript

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono gsfonts libcupsfilters1 libcupsimage2
  libgs9 libgs9-common libijs-0.35 libjbig2dec0 poppler-data
Suggested packages:
  fonts-noto ghostscript-x poppler-utils fonts-japanese-mincho
  | fonts-ipafont-mincho fonts-japanese-gothic | fonts-ipafont-gothic
  fonts-arphic-ukai fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  fonts-droid-fallback fonts-noto-mono ghostscript gsfonts libcupsfilters1
  libcupsimage2 libgs9 libgs9-common libijs-0.35 libjbig2dec0 poppler-data
0 upgraded, 11 newly installed, 0 to remove and 42 not upgraded.
Need to get 14.1 MB of archives.
After this operation, 49.9 MB of additional disk space will be used.
Get:1 http:

In [ ]:
!pip install ghostscript

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


* Barra de progresso que será exibida na transformação de pdf para .csv ↓

In [ ]:
!apt update

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [788 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [ ]:
!apt install ghostscript -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono gsfonts libcups2 libcupsfilters1
  libcupsimage2 libgs9 libgs9-common libijs-0.35 libjbig2dec0 poppler-data
Suggested packages:
  fonts-noto ghostscript-x cups-common poppler-utils fonts-japanese-mincho
  | fonts-ipafont-mincho fonts-japanese-gothic | fonts-ipafont-gothic
  fonts-arphic-ukai fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  fonts-droid-fallback fonts-noto-mono ghostscript gsfonts libcupsfilters1
  libcupsimage2 libgs9 libgs9-common libijs-0.35 libjbig2dec0 poppler-data
The following packages will be upgraded:
  libcups2
1 upgraded, 11 newly installed, 0 to remove and 76 not upgraded.
Need to get 14.3 MB of archives.
After t

In [ ]:
import zipfile
import os
import camelot 
import pandas as pd
import ghostscript


def progresso(atual, total):
    porcentagem = round(atual / (total / 100), 2)
    barrinha = '{:<50}'.format(int(porcentagem/2)*'❚')
    barra_comp = f'[{barrinha}] {porcentagem}% | [{atual}/{total}]'
    return barra_comp

* Extração do anexo 1 do anexos.zip que deve estar no ambiente de execução ↓

In [28]:
with zipfile.ZipFile('anexos.zip', 'r') as anexos_zip:
    anexo_1 = (anexos_zip.namelist())[0]
    anexos_zip.extract(anexo_1)

* O camelot irá ler as páginas especificadas nas variáveis 'pag_inicial' e 'pag_final' e transformar as tabelas em .csv, após cada leitura elas serão concatenadas num dataframe pandas.
* O máximo de otimização que eu consegui nessa parte foi com um 'for' para ler uma por uma, provavelmente há um modo mais eficiente.
* Em seguida as rows das colunas 'OD' e 'AMB' são alteradas no dataframe. Entendi essa tarefa de duas formas, explicitadas no código abaixo↓

In [29]:
master_df = pd.DataFrame()
pag_inicial = 3
pag_final = 200
ler = (pag_final-pag_inicial)+1
for n in range(ler):
    tabelas = camelot.read_pdf(f'{anexo_1}', process_background=True,
                               pages=f'{n + pag_inicial}', line_scale = 35)
    tabelas.export(f'tabela.csv', f='csv')
    master_df = pd.concat([master_df, pd.read_csv(f'tabela-page-{n + pag_inicial}-table-1.csv')])
    os.remove(f'tabela-page-{n + pag_inicial}-table-1.csv')
    print(f'Criando tabelas: {progresso(n, ler)}') #no google colab o "\r" para escrever em uma linha só não funcionou
print(f'Criando tabelas: {progresso(ler, ler)}')

# cria um novo pandas dataframe, itera sobre as rows e altera de acordo
master_df.to_csv('tabela_unica.csv', index=False)
df = pd.read_csv("tabela_unica.csv")
correct_df = df.copy()
for index, row in correct_df.iterrows():
    if row['OD'] == 'OD':
        correct_df.loc[index, 'OD'] = 'Seg. Odontológica'
    if row['AMB'] == 'AMB':
        correct_df.loc[index, 'AMB'] = 'Seg. Ambulatorial'

# entendi a tarefa de alterar as células de forma ambígua
# caso o correto seja alterar apenas as colunas, ative abaixo:
'''
correct_df.rename(columns={'OD': 'Seg. Odontológica', 'AMB': 'Seg. Ambulatorial'}, inplace=True)
'''

correct_df.to_csv(r'tabela_unica_fixd.csv', index=False, header=True)
with zipfile.ZipFile('Teste_Pedro_Mendes.zip', 'w', compression=zipfile.ZIP_DEFLATED) as zipado:
    zipado.write('tabela_unica_fixd.csv')
os.remove('tabela_unica.csv')
os.remove('tabela_unica_fixd.csv')
os.remove(f'{anexo_1}')

Criando tabelas: [                                                  ] 0.0% | [0/198]
Criando tabelas: [                                                  ] 0.51% | [1/198]
Criando tabelas: [                                                  ] 1.01% | [2/198]
Criando tabelas: [                                                  ] 1.52% | [3/198]
Criando tabelas: [❚                                                 ] 2.02% | [4/198]
Criando tabelas: [❚                                                 ] 2.53% | [5/198]
Criando tabelas: [❚                                                 ] 3.03% | [6/198]
Criando tabelas: [❚                                                 ] 3.54% | [7/198]
Criando tabelas: [❚❚                                                ] 4.04% | [8/198]
Criando tabelas: [❚❚                                                ] 4.55% | [9/198]
Criando tabelas: [❚❚                                                ] 5.05% | [10/198]
Criando tabelas: [❚❚                                  